# Spark

Hadoop Mapreduce nem elég rugalmas, túl sok kódot kell írni. Spark egy magasabb szintű megoldás az eloszott adatfeldolgozáshoz.

A spark legalapabb adatmanipuláló eszköze az RDD (Resilient Distributed Dataset).

Ha az RDD API-t akarjuk használni programunknak először egy konfigurációs és egy kontextus objektumot kell létrehoznia.

In [1]:
from pyspark import SparkConf, SparkContext 

conf = SparkConf() 
sc = SparkContext(conf=conf)

Az alábbi kódrészelettel ellenőrizhető, működik-e a spark.

In [2]:
x = "MWűGkyösdwinko Qau fSJpDaTrykv!w"

rdd = sc.parallelize([x[i:i+2] for i in range(0, len(x)-2, 2)], 2)
res = rdd.map(lambda item: item[0]).collect()

print("".join(res))

Működik a Spark


RDD-t több módon is létre tudunk hozni. Péládul meglévő adatból.

Egy RDD a klaszter több számítógépén elosztva kerül végrehajtásra, amiket partícióknak nevezünk.

In [3]:
numbers = [1, 2, 4, 6, 7, 2, 4, 6, 5]

rdd = sc.parallelize(numbers, 3)

print(numbers)
print(rdd)
print(rdd.getNumPartitions()) # partíciók száma

[1, 2, 4, 6, 7, 2, 4, 6, 5]
ParallelCollectionRDD[2] at readRDDFromFile at PythonRDD.scala:289
3


Ez a kódrészlet a numbers listából egy RDD-t készít, amit a klaszteren három gépen oszt el. Az RDD-t akár újra is tudjuk osztani

In [3]:
rdd2 = rdd.repartition(5)
rdd2.getNumPartitions()

5

## Műveletek
Az RDD-ken kétféle műveletet végezhetünk. A transzformációk valamilyen logika mentén egy új RDD-t hoznak létre. Az akciók pedig az RDD-ből valameilyen konrét adatot hoznak létre. Az RDD-n meghívott műveletlánc akkor értékelődik ki amikor akciót hívunk.

### Akciók
Fontos akciók a következők:

In [18]:
rdd.collect() # adatok összegyűjtése

['MW',
 'űG',
 'ky',
 'ös',
 'dw',
 'in',
 'ko',
 ' Q',
 'au',
 ' f',
 'SJ',
 'pD',
 'aT',
 'ry',
 'kv']

In [5]:
rdd.count() # adatok megszámlálása

9

In [6]:
rdd.take(5) # első n elem összegyűjtése 

[1, 2, 4, 6, 7]

In [7]:
numbers3 = sc.parallelize(range(5), 3)

A legfontosabb transzformációk a következők a filter, map, reduce

In [8]:
numbers3.take(10)

[0, 1, 2, 3, 4]

In [17]:
numbers3.getNumPartitions()

16

In [ ]:
# narrow transformation -> nem jön létre új partíció
# transzformáció lusta kiértékelésű, akció mohó

In [26]:
numbers3.filter(lambda number: number % 2 == 0).reduce(lambda acc, curr: acc + curr)

249500

In [4]:
text = sc.textFile('alkotmany.txt')

In [5]:
text

alkotmany.txt MapPartitionsRDD[8] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
text.take(2)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:',
 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.']

In [7]:
search_for = 'magyar'
text.filter(lambda line: search_for in line).collect()

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:',
 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.',
 'Büszkék vagyunk a magyar emberek nagyszerű szellemi alkotásaira.',
 'Ígérjük, hogy megőrizzük az elmúlt évszázad viharaiban részekre szakadt nemzetünk szellemi és lelki egységét. Kinyilvánítjuk, hogy a velünk élő nemzetiségek a magyar politikai közösség részei és államalkotó tényezők.',
 'Vállaljuk, hogy örökségünket, egyedülálló nyelvünket, a magyar kultúrát, a magyarországi nemzetiségek nyelvét és kultúráját, a Kárpát-medence természet adta és ember alkotta értékeit ápoljuk és megóvjuk. Felelősséget viselünk utódainkért, ezért anyagi, szellemi és természeti erőforrásaink gondos használatával védelmezzük az utánunk jövő nemzedékek életfeltételeit.',
 'Nem ismerjük el történeti alkotmányunk idegen megszállá

In [34]:
text.flatMap(lambda line: line.split(" ")).take(2)

['Mi,', 'a']

A `reduceByKey` az azonos kulcsú rekordokat redukálja.

In [39]:
text.flatMap(lambda line: line.split(" "))\
.map(lambda word: (word.replace(''), 1))\
.reduceByKey(lambda count, current: count + current)\
.sortBy(lambda t: t[1], False)\
.take(10)

[('a', 41),
 ('és', 24),
 ('az', 19),
 ('hogy', 17),
 ('valljuk,', 8),
 ('magyar', 6),
 ('nemzet', 5),
 ('szellemi', 4),
 ('büszkék', 4),
 ('vagyunk', 4)]